# MBTI별 포스팅 모으기 

In [154]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request

In [155]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# OverSampling and Undersampling (데이터 500개 정도로 맞추기)

In [156]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [157]:
X_data = mbti['posts']
X = X_data.values.reshape(-1, 1)

y_data = mbti['type']
y = y_data.values.reshape(-1, 1)

Counter(y_data)
X_resampled, y_resampled = RandomUnderSampler(random_state=0).fit_sample(X, y)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)

Counter({'ENFJ': 39,
         'ENFP': 39,
         'ENTJ': 39,
         'ENTP': 39,
         'ESFJ': 39,
         'ESFP': 39,
         'ESTJ': 39,
         'ESTP': 39,
         'INFJ': 39,
         'INFP': 39,
         'INTJ': 39,
         'INTP': 39,
         'ISFJ': 39,
         'ISFP': 39,
         'ISTJ': 39,
         'ISTP': 39})

# tokenize and stemming

In [158]:
X_data = X_resampled.ravel()
y_data = y_resampled


print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    rm_urls = re.sub(r'http[s]?://\S+', '', sentence)
    clean_sentence = re.sub('[^A-Za-z\s]+', '', rm_urls.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [159]:
import nltk
import gc
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [160]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stopwords = ["http*"]
# print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        tempData.append(ps.stem(word))
        tempData = [word for word in tempData if not word in stopwords] # 불용어 제거
    stemData.append(tempData)
print("stemming 완료")
print(stemData[1])

#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]
del [[mbti]]
gc.collect()

stemming 시작
stemming 완료
['sorri', 'i', 'know', 'thi', 'post', 'is', 'a', 'few', 'year', 'old', 'but', 'i', 'wa', 'so', 'happi', 'to', 'read', 'it', 'becaus', 'it', 'make', 'so', 'much', 'sens', 'to', 'me', 'especi', 'now', 'that', 'i', 'realiz', 'im', 'truli', 'an', 'enfj', 'and', 'not', 'an', 'esfj', 'i', 'wa', 'rais', 'byentp', 'for', 'sure', 'winkhi', 'i', 'liter', 'just', 'saw', 'thi', 'today', 'havent', 'been', 'on', 'perc', 'in', 'a', 'few', 'month', 'i', 'might', 'retir', 'i', 'just', 'dont', 'think', 'thi', 'forum', 'is', 'healti', 'for', 'me', 'unsurethi', 'wa', 'hilari', 'hahaha', 'blue', 'you', 'are', 'just', 'too', 'awesom', 'for', 'word', 'super', 'hugsi', 'wouldnt', 'say', 'bitch', 'but', 'you', 'do', 'seem', 'a', 'littl', 'selfcent', 'mayb', 'you', 'need', 'a', 'wake', 'up', 'call', 'no', 'offens', 'or', 'anythingmast', 'wolf', 'isnt', 'jealou', 'of', 'all', 'my', 'admir', 'becaus', 'we', 'are', 'just', 'friend', 'in', 'fact', 'he', 'like', 'a', 'brother', 'to', 'me', 'e

4674

In [161]:
#vectorization (tfidf)
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
X = tfidf.transform(stemData).toarray()
y = np.array(y_data)




[[0.11362201 0.         0.         ... 0.         0.         0.        ]
 [0.23669652 0.03730275 0.         ... 0.         0.         0.        ]
 [0.10176628 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.21520038 0.03391501 0.         ... 0.         0.         0.        ]
 [0.17876285 0.         0.         ... 0.         0.         0.        ]
 [0.23887051 0.         0.         ... 0.         0.         0.        ]]


# XGBoost_Oversampling

In [162]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [163]:
print("Model building Start")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)
model = XGBClassifier()
model.fit(X_train, y_train)

Model building Start


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [164]:
preds = model.predict(X_test)
print("Model building End")

Model building End


In [165]:
import joblib

joblib.dump(model, 'XGBoost_oversample.pkl')

['XGBoost_oversample.pkl']

In [166]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")
# https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/

Evaluation Start
0.4946808510638298
[0.75       0.45454545 0.54545455 0.16       1.         0.46153846
 0.61538462 0.71428571 0.4        0.72727273 0.375      0.375
 0.47368421 0.5        0.5        0.5       ]
[0.75       0.38461538 0.75       0.4        0.4375     0.54545455
 0.53333333 0.38461538 0.25       0.61538462 0.33333333 0.3
 0.6        0.5        0.55555556 0.54545455]
[0.75       0.41666667 0.63157895 0.22857143 0.60869565 0.5
 0.57142857 0.5        0.30769231 0.66666667 0.35294118 0.33333333
 0.52941176 0.5        0.52631579 0.52173913]
Evaluation END
